In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.utils.prune as prune
import os
import copy
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# 2. 유틸리티: 모델 사이즈(MB) 및 파라미터 계산 함수
def get_model_info(model, name="Model"):
    # 모델을 임시 저장하여 파일 크기 측정
    torch.save(model.state_dict(), "temp.p")
    size_mb = os.path.getsize("temp.p") / 1e6
    os.remove("temp.p")

    # 파라미터 수 계산 (0이 아닌 것만)
    total_params = sum(p.numel() for p in model.parameters())
    zero_params = sum((p == 0).sum().item() for p in model.parameters())
    non_zero_params = total_params - zero_params

    print(f"[{name}] Size: {size_mb:.2f} MB | Non-Zero Params: {non_zero_params:,} (Total: {total_params:,})")
    return size_mb, non_zero_params

# 3. 유틸리티: 정확도 평가 함수
def evaluate(model, loader, device=device):
    model.eval()
    correct = 0
    total = 0
    start_time = time.time()
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    inference_time = (time.time() - start_time) / len(loader.dataset) * 1000 # ms per sample (roughly)
    return acc, inference_time

In [ ]:
# Data Pipeline: CIFAR-10
# Data Normalization: CIFAR-10 데이터셋의 채널별 평균과 표준편차를 사용하여 정규화.
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

print("Loading CIFAR-10 Dataset...")
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Model Architecture: ResNet-18 for CIFAR-10
def get_resnet18_cifar():
    # 공식 torchvision 모델 로드 (Pretrained 아님)
    model = torchvision.models.resnet18(weights=None)

    # CIFAR-10 이미지 크기(32x32)에 맞춰 입력단 수정
    # 원본 ResNet은 224x224용이라 7x7 conv와 maxpool이 정보를 너무 많이 날림
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity() # Maxpool 제거
    model.fc = nn.Linear(512, 10) # 10개 클래스
    return model.to(device)

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

model = get_resnet18_cifar()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15, 25], gamma=0.1)

save_dir = "/"

# 체크포인트 경로
checkpoint_path = os.path.join(save_dir, 'checkpoint.pth')
best_model_path = os.path.join(save_dir, 'best_baseline.pth')

start_epoch = 0
best_acc = 0.0

# Training Loop: Resume Support
if os.path.exists(checkpoint_path):
    print(f"Checkpoint found. Resuming from {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state'])
    optimizer.load_state_dict(checkpoint['optimizer_state'])
    scheduler.load_state_dict(checkpoint['scheduler_state'])
    start_epoch = checkpoint['epoch'] + 1
    best_acc = checkpoint['best_acc']
    print(f"   Resuming from Epoch {start_epoch}, Current Best Acc: {best_acc:.2f}%")
else:
    print("Starting Training from Scratch...")

epochs = 30
print("-" * 60)
for epoch in range(start_epoch, epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    scheduler.step()

    current_acc, _ = evaluate(model, testloader)
    avg_loss = running_loss / len(trainloader)

    print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f} | Acc: {current_acc:.2f}%")

    # Best Model 저장
    if current_acc > best_acc:
        best_acc = current_acc
        torch.save(model.state_dict(), best_model_path)
        print(f"   New Best Model Saved! ({best_acc:.2f}%)")

    # Checkpoint 저장 (매 epoch)
    torch.save({
        'epoch': epoch,
        'model_state': model.state_dict(),
        'optimizer_state': optimizer.state_dict(),
        'scheduler_state': scheduler.state_dict(),
        'best_acc': best_acc
    }, checkpoint_path)

print("-" * 60)
print(f"Baseline Training Finished. Best Acc: {best_acc:.2f}%")

In [ ]:
# Baseline 로드.
baseline_model = get_resnet18_cifar()
baseline_model.load_state_dict(torch.load(best_model_path))
base_acc, base_time = evaluate(baseline_model, testloader)
base_size, base_nz = get_model_info(baseline_model, "Baseline")

print(f"\nExperiment: Fine-grained Analysis (Focus on 60%~80%)...")
print("-" * 80)
print(f"{'Ratio':^10} | {'Acc (%)':^10} | {'Drop (%)':^10} | {'Params (M)':^12} | {'Status'}")
print("-" * 80)

# Pruning 비율 정의
ratios = [
    0.50,
    0.60, 0.65,
    0.68, 0.70, 0.72, 0.74, 0.75, 0.76, 0.78,
    0.80, 0.82, 0.85
]

results = {'ratio': [], 'acc': [], 'params': []}
best_pruned_model = None
target_ratio = 0.0

for r in ratios:
    model_temp = copy.deepcopy(baseline_model)

    # Pruning 수행
    parameters_to_prune = []
    for module in model_temp.modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            parameters_to_prune.append((module, 'weight'))

    prune.global_unstructured(
        parameters_to_prune,
        pruning_method=prune.L1Unstructured,
        amount=r,
    )

    # 마스크 영구 적용
    for module, _ in parameters_to_prune:
        prune.remove(module, 'weight')

    # 평가
    acc, _ = evaluate(model_temp, testloader)
    _, nz_params = get_model_info(model_temp, name=f"Prune {r}")

    # 결과 저장
    results['ratio'].append(r)
    results['acc'].append(acc)
    results['params'].append(nz_params)

    drop = base_acc - acc

    # 상태 메시지
    if drop < 1.0: status = "Safe"
    elif drop < 2.0: status = "Caution" # 1~2% 떨어짐 (Fine-tuning으로 복구 가능성 높음)
    elif drop < 5.0: status = "Warning"
    else: status = "Collapse"

    print(f"{r*100:^9.0f}% | {acc:^10.2f} | {drop:^10.2f} | {nz_params/1e6:^12.2f} | {status}")

    # Best Case 선정 기준 수정:
    # "정확도 하락이 2.0% 미만인 것 중에서 가장 많이 깎은 것"
    # 이유: 뒤에 Fine-tuning(재활치료)이 있으므로, 2% 정도 까진 깎여도 복구 가능함.
    if drop < 2.0:
        best_pruned_model = copy.deepcopy(model_temp)
        target_ratio = r

print("-" * 80)
print(f"Best Pruning Ratio Selected: {target_ratio*100:.0f}%")

In [ ]:
# Fine-tuning Process
print(f"\nPhase 2.5: Fine-tuning Best Model (Ratio: {target_ratio*100:.0f}%)")
print("   -> 가지치기로 떨어진 성능을 5 Epoch 동안 짧게 재학습하여 복구합니다.")

# 1. Best Model 복제 (재학습용)
ft_model = copy.deepcopy(best_pruned_model).to(device)
ft_model.train() # 학습 모드 전환

# 2. 설정: 기존보다 낮은 Learning Rate
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(ft_model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
scheduler_ft = optim.lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)

# 3. Fine-tuning Loop (5 Epochs)
ft_epochs = 5
ft_losses = []

print("-" * 60)
for epoch in range(ft_epochs):
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer_ft.zero_grad()
        outputs = ft_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        # Pruning을 'remove'로 영구 적용한 경우, 0값이 미세하게 변할 수 있음.
        optimizer_ft.step()

        running_loss += loss.item()

    scheduler_ft.step()
    print(f"   [Fine-tune] Epoch {epoch+1}/{ft_epochs} | Loss: {running_loss/len(trainloader):.4f}")

# 4. Fine-tuning 후 평가
ft_acc, _ = evaluate(ft_model, testloader)
ft_size, _ = get_model_info(ft_model, "Fine-tuned")

# Result Comparison: Pruning 직후 vs Fine-tuning 후
# 아까 리스트에서 Best Ratio에 해당하는 Acc 찾기
best_idx = ratios.index(target_ratio)
pruned_acc = results['acc'][best_idx]

print("\nRecovery Report")
print(f"   - Before FT (Pruned): {pruned_acc:.2f}%")
print(f"   - After FT (Retrained): {ft_acc:.2f}%")
print(f"   - Recovered: +{ft_acc - pruned_acc:.2f}%p")

# PPT용 표 출력
print("\nPPT Table Data")
print("| State | Accuracy | Size (MB) |")
print("|-------|----------|-----------|")
print(f"| Baseline | {base_acc:.2f}% | {base_size:.2f} |")
print(f"| Pruned   | {pruned_acc:.2f}% | {base_size:.2f} (Sparse) |")
print(f"| Fine-tuned | {ft_acc:.2f}% | {base_size:.2f} (Sparse) |")

# Best Pruned Model을 Fine-tuning 된 놈으로 교체 (이걸로 Quantization 가야 함)
best_pruned_model = ft_model

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 1. Sparsity vs Accuracy 그래프 (Collapse Zone 표시)
plt.figure(figsize=(10, 6))
plt.plot(np.array(ratios)*100, results['acc'], 'r-o', linewidth=2, label='Pruned Accuracy')
plt.axhline(y=base_acc, color='b', linestyle='--', label=f'Baseline ({base_acc:.1f}%)')
# Fine-tuning 결과 점 찍기
plt.plot(target_ratio*100, ft_acc, 'g*', markersize=15, label=f'After Fine-tuning ({ft_acc:.1f}%)')

# Collapse Zone (정확도 10% 이상 떨어진 구간) 색칠
collapse_threshold = base_acc - 10
for i in range(len(ratios)-1):
    if results['acc'][i+1] < collapse_threshold:
        plt.axvspan(ratios[i]*100, 100, color='gray', alpha=0.2, label='Collapse Zone')
        plt.text(ratios[i]*100 + 2, collapse_threshold-5, "CRITICAL\nCOLLAPSE", color='red', fontweight='bold')
        break

plt.title(f'Sensitivity Analysis: When does the model break?', fontsize=14)
plt.xlabel('Sparsity (%)', fontsize=12)
plt.ylabel('Accuracy (%)', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('pruning_curve.png', dpi=150)
print("그래프 저장 완료: pruning_curve.png")

# 2. Kernel Visualization (가중치 0 된거 보여주기)
def plot_kernels(tensor, title):
    if tensor.dim() == 4: # Conv Layer
        weights = tensor.data.cpu().numpy()[:16] # 첫 16개만
        weights = np.mean(weights, axis=1) # 채널 평균
    else: return

    fig, axes = plt.subplots(2, 8, figsize=(16, 4))
    fig.suptitle(title, fontsize=16)
    for i, ax in enumerate(axes.flat):
        ax.imshow(weights[i], cmap='viridis')
        ax.axis('off')
    return fig

# Baseline vs Pruned 비교
fig1 = plot_kernels(baseline_model.conv1.weight, "Original Weights (Dense)")
plt.savefig('kernel_before.png', bbox_inches='tight')

# 주의: Fine-tuning 후에도 0이 유지되었는지 확인하는 용도
fig2 = plot_kernels(best_pruned_model.conv1.weight, f"Pruned Weights (Sparsity {target_ratio*100:.0f}%)")
plt.savefig('kernel_after.png', bbox_inches='tight')

print("커널 시각화 저장 완료: kernel_before.png, kernel_after.png")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_sparsity_comparison(model_orig, model_pruned, layer_name="layer2.0.conv1"):
    """
    특정 레이어의 가중치를 가져와서 0이 된 부분(Pruned)을 시각화합니다.
    """
    # 1. 레이어 객체 가져오기 (문자열 이름으로 접근)
    # 예: layer2의 0번째 블록의 conv1
    layer_orig = dict(model_orig.named_modules())[layer_name]
    layer_pruned = dict(model_pruned.named_modules())[layer_name]

    # 2. 가중치 추출 (CPU로 이동)
    w_orig = layer_orig.weight.data.cpu().numpy()
    w_pruned = layer_pruned.weight.data.cpu().numpy()

    # 3. 시각화를 위해 4D 텐서를 2D 행렬로 펼치기
    # (Out_Channel, In_Channel, K, K) -> (Out_Channel, Flattened_Input)
    # 이렇게 하면 가중치 전체 분포를 한눈에 볼 수 있습니다.
    w_orig_flat = w_orig.reshape(w_orig.shape[0], -1)
    w_pruned_flat = w_pruned.reshape(w_pruned.shape[0], -1)

    # 4. 희소성(Sparsity) 계산
    sparsity = 100 * np.sum(w_pruned == 0) / w_pruned.size

    # ================= 그리기 =================
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # [왼쪽] 원본 모델
    # 0이 아닌 값만 색칠 (대부분 꽉 차 있음)
    axes[0].imshow(np.abs(w_orig_flat), cmap='viridis', aspect='auto', vmin=0, vmax=0.1)
    axes[0].set_title(f"Baseline: {layer_name}\n(Dense, No Zeros)", fontsize=14)
    axes[0].set_xlabel("Input Connections (Flattened)")
    axes[0].set_ylabel("Output Filters")

    # [오른쪽] Pruned 모델 (핵심!)
    # 0인 부분은 '검은색'으로 나옴 -> 마스크 시각화
    # 값이 있으면 1(흰색/노란색), 0이면 0(검은색/보라색)
    mask = (w_pruned_flat != 0).astype(float)

    axes[1].imshow(mask, cmap='gray', aspect='auto', interpolation='nearest')
    axes[1].set_title(f"Pruned: {layer_name}\n(Sparsity: {sparsity:.2f}%)", fontsize=14, color='red')
    axes[1].set_xlabel("Input Connections (Flattened)")
    axes[1].set_yticks([]) # Y축 라벨 중복 제거

    plt.tight_layout()
    plt.show()

# --- 실행 ---
print("🔍 Visualizing Layer 2 (Pruning이 활발한 구간)...")
# layer1은 보통 중요해서 많이 남고, layer2나 layer3부터 많이 잘립니다.
visualize_sparsity_comparison(baseline_model, best_pruned_model, layer_name="layer2.0.conv1")

print("\n🔍 Visualizing Layer 3 (더 깊은 구간)...")
visualize_sparsity_comparison(baseline_model, best_pruned_model, layer_name="layer3.0.conv1")

In [ ]:
import torchvision
from torch.nn.quantized import FloatFunctional

# 1. 양자화 호환이 가능한 BasicBlock 정의
class QuantizableBasicBlock(nn.Module):
    def __init__(self, original_block):
        super(QuantizableBasicBlock, self).__init__()
        # 기존 가중치와 설정을 그대로 가져옵니다
        self.conv1 = original_block.conv1
        self.bn1 = original_block.bn1
        self.relu = original_block.relu
        self.conv2 = original_block.conv2
        self.bn2 = original_block.bn2
        self.downsample = original_block.downsample
        self.stride = original_block.stride

        # 덧셈을 위한 특수 객체 추가
        self.skip_add = FloatFunctional()

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        # += 연산자 대신 FloatFunctional.add 사용
        out = self.skip_add.add(out, identity)

        out = self.relu(out)
        return out

# 2. 모델 내의 모든 BasicBlock을 교체하는 함수
def replace_resnet_blocks(module):
    # 재귀적으로 모델 내부를 탐색하며 BasicBlock을 교체합니다
    for name, child in module.named_children():
        if isinstance(child, torchvision.models.resnet.BasicBlock):
            setattr(module, name, QuantizableBasicBlock(child))
        else:
            replace_resnet_blocks(child)

print("Patching ResNet blocks for Quantization...")

# 1. 모델 준비 (Wrapper 적용)
original_model = copy.deepcopy(best_pruned_model).to('cpu')
model_to_quantize = QuantizedModelWrapper(original_model)

# 2. 블록 교체 실행 (Prepare 전에)
replace_resnet_blocks(model_to_quantize.model_fp32)
print("ResNet BasicBlocks have been replaced with Quantizable versions.")

model_to_quantize.eval()
model_to_quantize.qconfig = torch.quantization.get_default_qconfig('fbgemm')

# 3. Fuse (선택 사항)

# 4. Prepare
print("   Preparing model...")
torch.quantization.prepare(model_to_quantize, inplace=True)

# 5. Calibration
print("   Running calibration (feed forward)...")
with torch.no_grad():
    for i, (inputs, _) in enumerate(trainloader):
        if i >= 50: break
        model_to_quantize(inputs.to('cpu'))

# 6. Convert
print("   Converting to INT8...")
model_int8 = torch.quantization.convert(model_to_quantize, inplace=True)

# 7. 평가
print("   Evaluating quantized model...")
q_acc, q_time = evaluate(model_int8, testloader, device='cpu')
q_size, _ = get_model_info(model_int8, "Quantized")

print(f"\nFinal Report")
print(f"1. Baseline:   Acc={base_acc:.2f}%, Size={base_size:.2f}MB")
print(f"2. Pruned:     Acc={results['acc'][ratios.index(target_ratio)]:.2f}%, Size={base_size:.2f}MB (Sparse)")
print(f"3. Quantized:  Acc={q_acc:.2f}%, Size={q_size:.2f}MB")
print(f" 최종 압축률: {base_size/q_size:.2f}배 감소")
print(f" 정확도 손실: {base_acc - q_acc:.2f}%")

In [ ]:
import os

# 저장 경로 설정
save_path = "resnet_pruned_quantized.pth"

# 모델 저장 (JIT Script 방식으로 저장하는 것이 호환성에 가장 좋습니다)
# 양자화 모델은 구조와 파라미터가 묶여있어서 torch.jit.script를 추천합니다.
print(f"Saving model to {save_path}...")

# 1. 간단한 저장 (State Dict)
torch.save(model_int8.state_dict(), save_path)

# 2. 파일 크기 확인
file_size = os.path.getsize(save_path) / (1024 * 1024)
print(f"Model saved. File Size on Disk: {file_size:.2f} MB")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

# 그래프 1: Accuracy vs Pruning Ratio
plt.subplot(1, 2, 1)
plt.plot(np.array(results['ratio'])*100, results['acc'], 'r-o', label='Pruned Acc')
plt.axhline(y=base_acc, color='b', linestyle='--', label='Baseline Acc')
plt.axvline(x=target_ratio*100, color='g', linestyle=':', label='Selected Best Point')

# 붕괴 지점 표시 (정확도가 10% 이상 떨어진 첫 지점)
for r, acc in zip(results['ratio'], results['acc']):
    if base_acc - acc > 10:
        plt.annotate('Collapse!', xy=(r*100, acc), xytext=(r*100, acc-10),
                     arrowprops=dict(facecolor='black', shrink=0.05))
        break

plt.title('Sensitivity Analysis: Accuracy vs Pruning')
plt.xlabel('Pruning Ratio (%)')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True)

# 그래프 2: Parameter Reduction
plt.subplot(1, 2, 2)
# 파라미터 수를 백만(M) 단위로 변환
params_m = np.array(results['params']) / 1e6
plt.plot(np.array(results['ratio'])*100, params_m, 'b-s', label='Active Params')
plt.title('Parameter Reduction')
plt.xlabel('Pruning Ratio (%)')
plt.ylabel('Parameters (Millions)')
plt.grid(True)

plt.tight_layout()
plt.savefig(os.path.join(save_dir, 'final_result_graph.png'))
plt.show()

print("그래프 생성 완료! 발표 자료에 사용하세요.")

In [ ]:
from google.colab import files
try:
    files.download('resnet_pruned_quantized.pth')
    print("다운로드 요청 완료. (브라우저 다운로드 창을 확인하세요)")
except Exception as e:
    print("다운로드 실패. 왼쪽 파일 탐색기에서 우클릭 -> 다운로드 하세요.")

In [ ]:
from google.colab import drive
import shutil
import os

# 1. 구글 드라이브 마운트 (연동)
print("Mounting Google Drive...")
drive.mount('/content/drive')

# 2. 저장할 경로 설정 (내 드라이브의 루트 경로)
# 원하시는 폴더가 있다면 '/content/drive/MyDrive/폴더명/' 으로 수정하세요.
destination_path = '/content/drive/MyDrive/resnet_pruned_quantized.pth'

# 3. 파일 복사 (현재 위치 -> 구글 드라이브)
source_path = 'resnet_pruned_quantized.pth' # 방금 저장한 모델 파일

if os.path.exists(source_path):
    shutil.copy(source_path, destination_path)
    print(f"구글 드라이브에 저장 완료. 경로: {destination_path}")

    # 잘 들어갔는지 파일 크기 확인
    size = os.path.getsize(destination_path) / (1024 * 1024)
    print(f"저장된 파일 크기: {size:.2f} MB")
else:
    print("모델 파일을 찾을 수 없습니다. 위에서 모델 저장을 먼저 실행해주세요.")